# Recognition of faces from Inception Resnet V1 ( Vedio Stream Version)

## Imports

In [1]:
import cv2
import numpy as np
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import numpy as np
import faiss
import os

## Face Detection through MTCNN

In [2]:
# Face detector
device='cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(keep_all=True, device=device)
device

'cpu'

## Face Recognition through Facenet pre-trained model

In [3]:
# Face recognition model (FaceNet pre-trained)
facenet = InceptionResnetV1(pretrained='vggface2').eval()

def extract_face(image, mtcnn, target_size=(160, 160)):
    """
    Detects and crops the face from the image.
    """
    boxes, _ = mtcnn.detect(image)
    if boxes is None:
        return None
    cropped_faces = []
    for box in boxes:
        x1, y1, x2, y2 = [int(b) for b in box]
        face = image[y1:y2, x1:x2]
        face_resized = cv2.resize(face, target_size)
        cropped_faces.append(face_resized)
    return cropped_faces

In [4]:
def encode_faces(faces, facenet):
    """
    Generates embeddings for detected faces.
    """
    embeddings = []
    for face in faces:
        # Convert to PyTorch tensor
        face = torch.tensor(face.transpose(2, 0, 1)).float().unsqueeze(0) / 255.0
        embedding = facenet(face).detach().numpy()
        embeddings.append(embedding)
    return np.vstack(embeddings)

### Building the database

In [5]:
def build_face_database(image_folder, mtcnn, facenet):
    """
    Creates a database of known faces from images in the given folder.
    """
    database = {}
    for file in os.listdir(image_folder):
        name, ext = os.path.splitext(file)
        if ext.lower() not in ['.jpg', '.png']:
            continue
        image = cv2.imread(os.path.join(image_folder, file))
        faces = extract_face(image, mtcnn)
        if faces:
            embeddings = encode_faces(faces, facenet)
            database[name] = embeddings[0]
    return database


## FAISS Search

In [6]:
def recognize_faces_faiss(input_image, database, mtcnn, facenet, threshold):
    """
    Recognizes faces in the input image by comparing with the database using FAISS.
    
    Parameters:
    - input_image: The image to process.
    - database: A dictionary of precomputed embeddings (name -> embedding).
    - mtcnn: The face detector.
    - facenet: The face recognition model.
    - threshold: Similarity threshold for recognition.
    
    Returns:
    - Annotated image with recognition results.
    """
    # Extract and crop faces
    faces = extract_face(input_image, mtcnn)
    if not faces:
        return input_image, []

    # Generate embeddings for detected faces
    embeddings = encode_faces(faces, facenet)
    
    # Prepare FAISS index
    # Convert database to a format suitable for FAISS
    db_embeddings = np.array(list(database.values())).astype('float32')
    db_names = list(database.keys())
    
    # Create FAISS index for L2 similarity (use IndexFlatIP for cosine similarity)
    index = faiss.IndexFlatIP(db_embeddings.shape[1])  # Inner Product for cosine similarity
    faiss.normalize_L2(db_embeddings)  # Normalize embeddings for cosine similarity
    index.add(db_embeddings)

    
    # Normalize face embeddings for cosine similarity
    faiss.normalize_L2(embeddings)
    
    # Search database for nearest matches
    distances, indices = index.search(embeddings, 1)  # Top-1 match for each embedding

    results = []
    for dist, idx in zip(distances[:, 0], indices[:, 0]):
        if dist > threshold and idx < len(db_names):
            results.append((db_names[idx], dist))
        else:
            results.append((None, dist))
    
    # Annotate image
    annotated_image = input_image.copy()
    for (box, (name, score)) in zip(mtcnn.detect(input_image)[0], results):
        x1, y1, x2, y2 = [int(b) for b in box]
        label = f"{name} ({score:.2f})" if name else "Unknown"
        color = (0, 255, 0) if name else (0, 0, 255)
        cv2.rectangle(annotated_image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(annotated_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.25, color, 1)
    
    return annotated_image, results

In [7]:
def recognize_faces_video(video_path, database, mtcnn, facenet, threshold=0.6):
    """
    Recognizes faces in a video by comparing with the database.
    """
    try:
        # Open video capture
        cap = cv2.VideoCapture(video_path if video_path else 0)
        if not cap.isOpened():
            raise ValueError("Failed to open video source.")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret or frame is None:
                print("End of video or failed to capture frame.")
                break

            try:
                # Call the face recognition function
                annotated_image_faiss, results_faiss = recognize_faces_faiss(frame, database, mtcnn, facenet, threshold)
                if annotated_image_faiss is None:
                    continue

                # Display the annotated frame
                cv2.imshow("Face Recognition", annotated_image_faiss)

            except Exception as e:
                continue

            # Press 'q' to exit the video loop
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        # Release resources and close windows
        cap.release()
        cv2.destroyAllWindows()
        for _ in range(1, 5):  # Ensure all windows are destroyed
            cv2.waitKey(1)


In [8]:
# Load known faces
database = build_face_database("known_faces", mtcnn, facenet)
print(database.keys)

<built-in method keys of dict object at 0x2a4053540>


## Runner Code

In [ ]:
# Recognize faces from video or webcam
video_path = None  # Replace with video path or leave as None for webcam
recognize_faces_video(video_path, database, mtcnn, facenet)

2024-12-11 14:42:36.220 Python[22818:19962132] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


In [ ]:
import cv2
import numpy as np
import nmslib

def recognize_faces_nmslib(input_image, database, mtcnn, facenet, threshold):
    """
    Recognizes faces in the input image by comparing with the database using NMSLIB.
    
    Parameters:
    - input_image: The image to process.
    - database: A dictionary of precomputed embeddings (name -> embedding).
    - mtcnn: The face detector.
    - facenet: The face recognition model.
    - threshold: Similarity threshold for recognition.
    
    Returns:
    - Annotated image with recognition results.
    """
    # Extract and crop faces
    faces = extract_face(input_image, mtcnn)
    if not faces:
        return input_image, []

    # Generate embeddings for detected faces
    embeddings = encode_faces(faces, facenet)
    
    # Prepare NMSLIB index
    # Convert database to a format suitable for NMSLIB
    db_embeddings = np.array(list(database.values())).astype('float32')
    db_names = list(database.keys())

    # Create NMSLIB index
    index = nmslib.init(method='hnsw', space='cosinesimil')
    for idx, embedding in enumerate(db_embeddings):
        index.addDataPoint(idx, embedding)
    index.createIndex({'post': 2}, print_progress=False)

    results = []
    for embedding in embeddings:
        # Query the nearest neighbor
        nearest_neighbors = index.knnQuery(embedding, k=1)
        idx, dist = nearest_neighbors[0][0], 1 - nearest_neighbors[1][0]  # Convert to similarity score
        if dist > threshold and idx < len(db_names):
            results.append((db_names[idx], dist))
        else:
            results.append((None, dist))
    
    # Annotate image
    annotated_image = input_image.copy()
    for (box, (name, score)) in zip(mtcnn.detect(input_image)[0], results):
        x1, y1, x2, y2 = [int(b) for b in box]
        label = f"{name} ({score:.2f})" if name else "Unknown"
        color = (0, 255, 0) if name else (0, 0, 255)
        cv2.rectangle(annotated_image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(annotated_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.25, color, 1)
    
    return annotated_image, results
